In [ ]:
# 🎛️ Notebook 02 — Feature Extraction for Speech Emotion Recognition

**Speech Emotion Recognition | PBL Project — Semester 2**

---

## Goals
1. Extract **MFCCs** (Mel-Frequency Cepstral Coefficients) from audio
2. Extract additional features: **pitch**, **energy**, **ZCR**
3. Visualize MFCC heatmaps
4. Build the **full feature matrix** from a dataset subset
5. Explore feature distributions across emotion classes

---
> This notebook assumes you've run Notebook 01 and understand waveforms/spectrograms.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from glob import glob

from src.audio_processing import AudioProcessor
from src.feature_extraction import FeatureExtractor

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

# RAVDESS emotion code → name
EMOTION_MAP = {
    '01': 'neutral', '02': 'calm', '03': 'happy',
    '04': 'sad',     '05': 'angry', '06': 'fearful',
    '07': 'disgust', '08': 'surprised'
}
EMOTIONS_TO_USE = ['neutral', 'happy', 'sad']  # mid-term subset

print('✅ Imports OK')

## 1. What Are MFCCs?

**MFCCs (Mel-Frequency Cepstral Coefficients)** are the most widely used features for speech and audio tasks.

### How they're computed:
```
Audio signal
    → Pre-emphasis (boost high freq)
    → Frame into short windows (~25ms)
    → Apply Hamming window
    → FFT → power spectrum
    → Map to Mel filterbanks (40 bins)
    → Log of filterbank energies
    → DCT → MFCC coefficients (typically 13–40)
```

The result: a **matrix of shape (n_mfcc, n_frames)** that captures *how* the voice sounds over time.

In [ ]:
# -----------------------------------------------------------------------
# Load one sample file for demonstration
# -----------------------------------------------------------------------
SAMPLE_WAV = '../data/ravdess_subset/Actor_01/03-01-03-01-01-01-01.wav'

proc = AudioProcessor(sample_rate=22050, duration=3.0)
ext  = FeatureExtractor(n_mfcc=40)

if os.path.exists(SAMPLE_WAV):
    signal, sr = proc.full_preprocess(SAMPLE_WAV)
    print(f'Loaded: {SAMPLE_WAV}')
else:
    print('⚠️  Sample file not found — generating synthetic signal for demo')
    sr = 22050
    t  = np.linspace(0, 2, 2 * sr)
    signal = 0.5 * np.sin(2 * np.pi * 200 * t) * np.exp(-t)
    signal = proc.normalize(signal)

print(f'Signal duration : {len(signal)/sr:.2f}s')
print(f'Sample rate     : {sr} Hz')

## 2. Compute & Visualize MFCCs

In [ ]:
# Raw MFCC matrix (40 coefficients x n_frames)
mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40)

print(f'MFCC matrix shape: {mfcc.shape}  (n_mfcc × n_frames)')

fig, ax = plt.subplots(figsize=(12, 4))
img = librosa.display.specshow(mfcc, x_axis='time', sr=sr,
                                cmap='coolwarm', ax=ax)
fig.colorbar(img, ax=ax)
ax.set_ylabel('MFCC Coefficient #')
ax.set_title('MFCC Heatmap (40 coefficients)')
plt.tight_layout()
plt.show()

## 3. From Matrix → Feature Vector

Classifiers need a **fixed-length 1-D vector** per audio clip.  
We summarize each of the 40 coefficients with **mean** and **std** across time frames.

```
MFCC matrix  (40 × n_frames)
    → mean per row  → 40 values
    → std  per row  → 40 values
    → concat        → 80-dimensional vector
```

In [ ]:
mfcc_mean = np.mean(mfcc, axis=1)   # shape: (40,)
mfcc_std  = np.std(mfcc,  axis=1)   # shape: (40,)
mfcc_feat = np.concatenate([mfcc_mean, mfcc_std])  # shape: (80,)

print(f'Mean vector shape : {mfcc_mean.shape}')
print(f'Std  vector shape : {mfcc_std.shape}')
print(f'Combined feature  : {mfcc_feat.shape}')
print(f'First 5 means     : {mfcc_mean[:5].round(3)}')

# Visualize mean MFCC profile
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))

ax1.bar(range(40), mfcc_mean, color='steelblue', alpha=0.8)
ax1.set_xlabel('MFCC Coefficient #')
ax1.set_ylabel('Mean Value')
ax1.set_title('Mean MFCC Profile')

ax2.bar(range(40), mfcc_std, color='coral', alpha=0.8)
ax2.set_xlabel('MFCC Coefficient #')
ax2.set_ylabel('Std Dev')
ax2.set_title('MFCC Standard Deviation Profile')

plt.tight_layout()
plt.show()

## 4. MFCC Delta Features

**Delta** = first derivative of MFCCs (captures rate of change)  
**Delta-Delta** = second derivative (captures acceleration)

Using all three (MFCC + Δ + ΔΔ) gives a **richer, time-aware** feature set.

In [ ]:
delta   = librosa.feature.delta(mfcc)
delta2  = librosa.feature.delta(mfcc, order=2)

fig, axes = plt.subplots(3, 1, figsize=(12, 8))
for ax, feat, title, cmap in zip(
    axes,
    [mfcc, delta, delta2],
    ['MFCC', 'MFCC Delta (1st order)', 'MFCC Delta-Delta (2nd order)'],
    ['coolwarm', 'RdYlBu', 'PuOr']
):
    librosa.display.specshow(feat, x_axis='time', sr=sr, cmap=cmap, ax=ax)
    ax.set_title(title)

plt.suptitle('MFCC + Delta Features', fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

# Full delta feature vector
all_feat   = np.vstack([mfcc, delta, delta2])  # (120, n_frames)
delta_feat = np.concatenate([np.mean(all_feat, axis=1),
                               np.std(all_feat, axis=1)])  # (240,)
print(f'Delta feature vector size: {delta_feat.shape[0]}')

## 5. Other Audio Features

In [ ]:
# --- Pitch (F0) ---
f0 = librosa.yin(signal, fmin=50, fmax=500, sr=sr)
f0_clean = f0[f0 > 0]  # remove unvoiced (silence) frames

fig, axes = plt.subplots(1, 3, figsize=(14, 3))

# Pitch over time
t_f0 = np.linspace(0, len(signal)/sr, len(f0))
axes[0].plot(t_f0, f0, color='purple', linewidth=0.8)
axes[0].set_title('Pitch (F0) over Time')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Frequency (Hz)')

# RMS Energy
rms = librosa.feature.rms(y=signal)[0]
t_rms = np.linspace(0, len(signal)/sr, len(rms))
axes[1].plot(t_rms, rms, color='darkorange', linewidth=0.8)
axes[1].set_title('RMS Energy over Time')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Energy')

# Zero Crossing Rate
zcr = librosa.feature.zero_crossing_rate(y=signal)[0]
t_zcr = np.linspace(0, len(signal)/sr, len(zcr))
axes[2].plot(t_zcr, zcr, color='teal', linewidth=0.8)
axes[2].set_title('Zero Crossing Rate over Time')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('ZCR')

plt.tight_layout()
plt.show()

print(f'Pitch stats  — mean: {np.mean(f0_clean):.1f} Hz, std: {np.std(f0_clean):.1f} Hz')
print(f'Energy stats — mean: {np.mean(rms):.4f}, std: {np.std(rms):.4f}')
print(f'ZCR stats    — mean: {np.mean(zcr):.4f}, std: {np.std(zcr):.4f}')

## 6. Full Feature Vector via FeatureExtractor Class

In [ ]:
ext = FeatureExtractor(n_mfcc=40)

features = ext.extract_all(signal, sr,
                             use_delta=True,
                             use_pitch=True,
                             use_energy=True,
                             use_zcr=True)

print(f'Full feature vector shape : {features.shape}')
print(f'\nBreakdown:')
print(f'  MFCC + delta + delta2 (mean+std) : 2 × 3 × 40 = 240')
print(f'  Pitch (mean, std, median)         :  3')
print(f'  Energy (mean, std)                :  2')
print(f'  ZCR (mean, std)                   :  2')
print(f'  Total                             : 247')

# Quick sanity check
assert features.shape[0] == 247, f'Unexpected shape: {features.shape}'
print('\n✅ Feature vector shape verified: 247')

## 7. Build Full Feature Matrix From Dataset

Extract features from every audio file in the dataset and save as a CSV.

In [ ]:
DATA_DIR = '../data/ravdess_subset'
EMOTIONS_TO_USE = ['neutral', 'happy', 'sad']

def get_emotion_from_filename(filename):
    """Extract emotion label from RAVDESS filename."""
    parts = os.path.basename(filename).replace('.wav','').split('-')
    if len(parts) < 7:
        return None
    return EMOTION_MAP.get(parts[2])

# Collect all .wav files
wav_files = []
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.endswith('.wav'):
            wav_files.append(os.path.join(root, f))

print(f'Found {len(wav_files)} .wav files in {DATA_DIR}')

if len(wav_files) == 0:
    print('\n⚠️  No files found. Simulating a small dataset for demo...')
    # Simulate 15 samples (5 per emotion) with random features
    np.random.seed(42)
    n_emotions = len(EMOTIONS_TO_USE)
    rows = []
    for i, emo in enumerate(EMOTIONS_TO_USE):
        for j in range(5):
            # Simulate slightly different feature distributions per emotion
            fake_feat = np.random.randn(247) + i * 0.5
            row = {'label': emo}
            for k, v in enumerate(fake_feat):
                row[f'feat_{k}'] = round(v, 4)
            rows.append(row)
    df = pd.DataFrame(rows)
    print(f'  Simulated DataFrame: {df.shape}')
else:
    proc = AudioProcessor(sample_rate=22050, duration=3.0)
    ext  = FeatureExtractor(n_mfcc=40)
    rows = []
    skipped = 0

    for fpath in tqdm(wav_files, desc='Extracting features'):
        emotion = get_emotion_from_filename(fpath)
        if emotion not in EMOTIONS_TO_USE:
            continue
        try:
            sig, sr_ = proc.full_preprocess(fpath)
            feat = ext.extract_all(sig, sr_)
            row  = {'label': emotion}
            for k, v in enumerate(feat):
                row[f'feat_{k}'] = round(v, 4)
            rows.append(row)
        except Exception as e:
            skipped += 1

    df = pd.DataFrame(rows)
    print(f'\nDataFrame shape: {df.shape}')
    print(f'Skipped: {skipped}')

print('\nClass distribution:')
print(df['label'].value_counts())

In [ ]:
# Save feature matrix
os.makedirs('../outputs', exist_ok=True)
df.to_csv('../outputs/features.csv', index=False)
print('Features saved → ../outputs/features.csv')
df.head(3)

## 8. Feature Distribution by Emotion Class

Visualise whether MFCC-1 and MFCC-2 values differ by emotion — this tells us if features are useful.

In [ ]:
# Check first 2 MFCC mean features (feat_0, feat_1)
feature_cols = [c for c in df.columns if c.startswith('feat_')]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

colors = {'neutral': 'steelblue', 'happy': 'seagreen', 'sad': 'crimson'}

for ax, feat in zip(axes, ['feat_0', 'feat_1']):
    for emo in EMOTIONS_TO_USE:
        subset = df[df['label'] == emo][feat]
        subset.plot.kde(ax=ax, label=emo, color=colors.get(emo, 'gray'), linewidth=2)
    ax.set_title(f'Distribution of {feat} (MFCC coeff) by Emotion')
    ax.set_xlabel('Feature Value')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()
print('If distributions overlap a lot → features are not very discriminative.')
print('If they are separated → the model can use them to distinguish emotions.')

In [ ]:
# Correlation heatmap of first 20 features
fig, ax = plt.subplots(figsize=(10, 8))
subset_df = df[feature_cols[:20]]
sns.heatmap(subset_df.corr(), cmap='coolwarm', center=0, ax=ax,
            linewidths=0.3, annot=False)
ax.set_title('Feature Correlation Matrix (first 20 MFCC features)')
plt.tight_layout()
plt.show()

## 9. Pipeline Summary

```
[ .wav file ]
    ↓  AudioProcessor.full_preprocess()
[ signal (1-D array) ]  trim silence → normalize → pre-emphasis
    ↓  FeatureExtractor.extract_all()
[ 247-dim feature vector ]
    → MFCC (40 coeff) + Δ + ΔΔ → mean+std = 240
    → Pitch (mean, std, median)  = 3
    → RMS Energy (mean, std)     = 2
    → ZCR (mean, std)            = 2
    ↓  EmotionClassifier.train(X, y)
[ Trained Model ]
    ↓  clf.predict(X_test)
[ Emotion label: 'happy' / 'sad' / 'neutral' ]
```

👉 **Next:** `src/train_eval.py` — training & evaluation

---
*PBL Project | Speech Emotion Recognition | Semester 2*